# Imports & Properties / IMF creation

In [ ]:
"""
The dowload links on the website do not work
Stellar Evolution and Atmosphere Models Dowload Link: https://w.astro.berkeley.edu/~jlu/spisea/ 

When setting up the spisea folders during the installation process I could not get the paths to PYSYN_CDBS and SPISEA_MODELS
set propertly (through editing my .bash_profile), so as a work around I just defined them here
"""

import os
os.environ["PYSYN_CDBS"] = "/Users/joesum/cdbs/grp/redcat/trds"
os.environ['SPISEA_MODELS'] = "/Users/joesum/cdbs/grp/redcat/trds" 


# Import other necessary packages. 
from spisea import synthetic, evolution, atmospheres, reddening, ifmr
from spisea.imf import imf, multiplicity
import numpy as np
import pdb
import matplotlib.pyplot as plt
from astropy.table import unique
import pickle 
import re

In [ ]:
"""Setting evolution model and cluster parameters"""
evo_model = evolution.MISTv1()
atm_func = atmospheres.get_merged_atmosphere
red_law = reddening.RedLawHosek18b()

distance = 10                                       # distance to the observed cluster measured in pc
extinction = 0                                      # takes extinction in Ks filter in magnitudes
metallicity_bins = [5*i/10 for i in range(-5,2)]    # -2.5 and +0.5 are the min and max metalicity values allowed by spisea
age_bins = np.log10([10**i for i in range(6,11)])   # 1e6 seems to be the smallest age allowed 
mass_bins = [10**i for i in range(6)]               # Anything above 10,000 solar masses crashes my kernel 

# Filters in the format needed by the pysynphot package (some filters - like 2MASS - may need to be downloaded)
# Details here: https://pysynphot.readthedocs.io/en/latest/appendixb.html 
hubble = ["wfc3,ir,f127m", "wfc3,ir,f139m", "wfc3,ir,f153m"]                       # Hubble Filters
johnson = ["johnson,u", "johnson,b", "johnson,v", "johnson,r", "johnson,i", "johnson,j", "johnson,k"] # Johnson
two_mass = ["2mass,j", "2mass,h", "2mass,ks"] #jhk2mass filters
sdss = ["sdss,u", "sdss,g", "sdss,r", "sdss,i", "sdss,z", ] #sdss filters 
filter_list = johnson + sdss 


"""Creating the initial mass fuction"""
mass_limits = np.array([1e-2, 1e2])
powers = np.array([-2.35]) # Salpeter function
multiplicity = None
# The standard IMF function didnt seem to work for me, so the broken powerlaw with only a single power has the same effect.
# Not sure what the error was
initial_mass_func = imf.IMF_broken_powerlaw(mass_limits, powers, multiplicity)

print(metallicity_bins, len(metallicity_bins))
print(age_bins, len(age_bins))
print(mass_bins, len(mass_bins))

# Create isochrone and both clusters for all age / metalicity combinations

In [ ]:
"""
Sometimes you will get an error that IsochronePhot doesnt have a spec_list parameter even though it does...
Not sure why... Just resetting the kernel normally fixed it for me though!

Also the output everytime an isochrone is generated is quite long; there is probably a way to stop this printing by editing
the original code, but I have just been collapsing the output in my notebook 
"""

    """Files saved with sytax:
        iso =   isochrone
        uc  =   unresolved cluster
        rc  =   resolved cluster
        a{} =   log(age) of cluster
        m{} =   10 * metalicity of cluster"""

# Create isochrones with all combinations of metalicity and age
for metallicity in metallicity_bins:
    for age in age_bins:
        print("iso_a{:.0f}_m{:.0f}".format(age, metallicity*10))
        
        temp_isochrone = synthetic.IsochronePhot( 
            age,
            extinction,
            distance,
            metallicity,
            filters=filter_list,
            evo_model=evo_model,
            atm_func=atm_func,
            red_law=red_law,
        );
        
        temp_file = open("clusters/isochrones/iso_a{:.0f}_m{:.0f}.obj".format(age, metallicity*10), "wb")
        pickle.dump(temp_isochrone, temp_file)



In [ ]:
# Set mass to be used in IMF (same for all)
mass = 10_000
seed = None # setting seed to None will cause consitent outputs everytime 

#for each isochrone create both a resolved and unresolved cluster
for file_name in os.listdir("clusters/isochrones"):
    nums = [int(i) for i in re.findall('[0-9]+', file_name)]
    age, met = nums[0], nums[1]
    iso = pickle.load(open(f"clusters/isochrones/{file_name}", "rb"))
    
    unresolved = synthetic.UnresolvedCluster(
    iso, initial_mass_func, mass, [3000, 52000], verbose=False)
    temp_file = open("clusters/unresolved/uc_a{:.0f}_m{:.0f}.obj".format(age, met*10), "wb")
    pickle.dump(unresolved, temp_file)

    resolved = synthetic.ResolvedCluster(
    iso, initial_mass_func, mass, seed=seed)
    temp_file = open("clusters/resolved/rc_a{:.0f}_m{:.0f}.obj".format(age, met*10), "wb")
    pickle.dump(resolved, temp_file)
  

# Generate Unresolved Clusters for age = 1 / Gyr M/H = 0 with varying mass

In [ ]:
# Load isochrone of age=1Gyr and metalicity 0
iso = pickle.load(open("clusters/isochrones/iso_a9_m0.obj", "rb"))

for mass in mass_bins:
    temp_unres_clust = synthetic.UnresolvedCluster(
        iso, initial_mass_func, mass, [100, 7500], verbose=False)
    temp_file = open("clusters/mass-dependance/mass{:.0f}.obj".format(np.log10(mass)),"wb")
    pickle.dump(temp_unres_clust, temp_file)


## Plot mass dependance of unresolved cluster output

In [ ]:
for file_name in os.listdir("clusters/mass-dependance"):
        unresolved_cluster = pickle.load(open("clusters/mass-dependance/{}".format(file_name), "rb"))
        plt.plot(unresolved_cluster.wave_trim*10**-4,
                unresolved_cluster.spec_trim,#*unresolved_cluster.wave_trim,
                label=file_name, linewidth=2)
        plt.xlabel('Wavelength ($\mu$m)', fontsize=24)
        plt.ylabel('log(F$_{\lambda}$)', fontsize=24) #'log($\lambda$F$_{\lambda}$)'
        plt.tick_params(axis='both', labelsize=20)
        plt.legend()
        plt.xlim(0.25,5.0)
        plt.gca().set_yscale('log')
        plt.rcParams["figure.figsize"] = (9,9)

plt.show()